In [1]:
import snake, queue, random, threading, math
import tensorflow as tf
import numpy as np
import tkinter as tk

In [2]:
stack_size = 4

In [3]:
class experience:
    states = None
    action = None
    reward = None
    transitions = None
    
    def __init__(self, states, action, reward, transitions):
        self.states = states
        self.action = action
        self.reward = reward
        self.transitions = transitions

In [4]:
# Q-Network
q = tf.keras.Sequential()
q.add(tf.keras.layers.Conv2D(15, (4, 4),
                             activation="relu", input_shape=(15, 15, stack_size)))
q.add(tf.keras.layers.Dense(32, activation="relu"))
q.add(tf.keras.layers.Dense(4, activation="relu"))
q.compile(optimizer="Adam", loss="mse")

# Replay Memory
replay_memory = np.empty([2000], dtype=experience)

In [5]:
class agent:
    directions = ["UP", "DOWN", "LEFT", "RIGHT"]
    phi = queue.deque()

    def __init__(self, game, return_queue, replay_memory, q, epsilon, discount, rate, batch_size):
        self.game = game
        self.return_queue = return_queue
        self.replay_memory = replay_memory
        self.q = q

        self.epsilon = epsilon
        self.discount = discount
        self.rate = rate
        self.batch_size = batch_size

    def epsilon_action(self):
        if random.randrange(0, 1) <= self.epsilon:
            action = self.directions[random.randint(0, 4)]
        else:
            action = self.directions[np.argmax(q.predict(self.phi))]
        return action

    def step(self):
        action = self.epsilon_action()
        self.game.step(action)

        while self.return_queue.empty():
            if not self.game.running:
                break

        state_reward = return_queue.get()

        phi_last = np.array(self.phi)
        self.phi.appendleft(state_reward[0])

        phi_current = np.array(self.phi)

        if len(self.phi) >= stack_size:
            self.phi.pop()
            self.replay_memory.append(experience(
                phi_last, action, state_reward[1], phi_current))

    def get_batch(self):
        batch = np.empty([self.batch_size], dtype=experience)
        for x in range(self.batch_size):
            batch[x] = replay_memory[random.randint(0, len(replay_memory))].states

        return batch

    def loss(self, phi, reward, action):
        q_of_phi = self.q.predict(phi)
        yj = reward + self.discount * np.amax(q_of_phi)
        return math.pow(yj - q_of_phi[self.directions.index(action)], 2)

    def learn(self):
        # This probably isn't correct but might as well try it
        q.train_on_batch(self.get_batch())



In [6]:
def train_agent(agent, epoch):
    print("Training epoch " + str(epoch) + ".")

    while agent.game.running:
        print("stepping")
        agent.step()
        print("learning")
        agent.learn()

    print("Training ended, agent scored " + str(game.score) + " points.")

In [7]:
game = snake.game(queue.Queue(1))
for x in range(100):
    return_queue = queue.Queue(1)

    dqn = agent(game, return_queue, replay_memory, q, 0.1, 0.95, 0.1, 10)

    training_thread = threading.Thread(target=train_agent, args=(dqn, x))
    training_thread.start()

    game.start(return_queue)
    training_thread.join()

game.w.destroy()

Training epoch 0.
stepping
learning
Exception in thread Thread-4:
Traceback (most recent call last):
  File "C:\Users\jason\anaconda3\envs\MLSnake\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\jason\anaconda3\envs\MLSnake\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-89139a0e9950>", line 8, in train_agent
    agent.learn()
  File "<ipython-input-5-ef7105934602>", line 57, in learn
    q.train_on_batch(self.get_batch())
  File "<ipython-input-5-ef7105934602>", line 46, in get_batch
    batch[x] = replay_memory[random.randint(0, len(replay_memory))].states
AttributeError: 'NoneType' object has no attribute 'states'

